In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
employment = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/ChildCareDeserts_Data 2/employment_rate.csv")
incomes = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/ChildCareDeserts_Data 2/avg_individual_income.csv")

In [3]:
#sort high demand and normal demand areas
hd_emp_mask = employment['employment rate'] > .60
hd_emp = employment[hd_emp_mask]

hd_inc_mask = incomes['average income'] < 60000
hd_inc = incomes[hd_inc_mask]

In [4]:
#now make lists of high and normal demand zip codes
#assuming each dataset has every zip code in it

#collecting high demand
hd_zips = hd_emp["zipcode"].to_list()
for r in hd_inc['ZIP code'].to_list():
    hd_zips.append(r)
#collecting normal demand
    
normal_zips = [] 
for n in incomes["ZIP code"].to_list():
    if n not in hd_zips:
        normal_zips.append(n)


In [5]:
#find all zip codes that require expansion/addition, ignore rest
existing = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/ChildCareDeserts_Data 2/child_care_regulated.csv")

In [6]:
# --- 1. Check capacity consistency ---
age_sum = (
    existing['infant_capacity'].fillna(0) +
    existing['toddler_capacity'].fillna(0) +
    existing['preschool_capacity'].fillna(0) +
    existing['school_age_capacity'].fillna(0)
)

# Difference between computed sum and reported total
existing['capacity_diff'] = age_sum - existing['total_capacity']

# Find facilities with mismatch
mismatch = existing[existing['capacity_diff'] != 0]
print("Facilities with inconsistent capacity totals:")
print(mismatch[['facility_id','zip_code','capacity_diff']])

# Optional: quick check summary
print("Number of mismatches:", len(mismatch))
print(existing['capacity_diff'].describe())

overflow_count = 0
for u in existing['capacity_diff'].to_list():
    if u > 0 :
        overflow_count+=1
print("Num facilities where more kids than slots = " , overflow_count)

Facilities with inconsistent capacity totals:
       facility_id  zip_code  capacity_diff
0             2416     13323             -6
1             5555     14701             -6
2             9066     12590             -6
6            51349     10474             -6
8           108407     11225            -12
...            ...       ...            ...
15599       892735     10462            -12
15600       897263     11205            -10
15601       901966     10705            -12
15602       892455     10303            -12
15603       911779     11436            -12

[10776 rows x 3 columns]
Number of mismatches: 10776
count    15604.000000
mean        -7.304281
std          5.317957
min        -12.000000
25%        -12.000000
50%        -12.000000
75%          0.000000
max          0.000000
Name: capacity_diff, dtype: float64
Num facilities where more kids than slots =  0


In [7]:
print(existing["total_capacity"])

0          8
1          8
2          8
3        120
4         75
        ... 
15599     16
15600     10
15601     16
15602     16
15603     16
Name: total_capacity, Length: 15604, dtype: int64


In [14]:
reg = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/ChildCareDeserts_Data 2/child_care_regulated.csv")

long_count = 0 
l = [] 
for zip in reg['zip_code']:
    if len(str(zip)) > 5:
        long_count += 1
        l.append(str(zip))
print(long_count)
print(l)

439
['117472023', '148269684', '138204015', '146085238', '144681246', '146072121', '120951224', '117682536', '145278745', '146093633', '149051400', '146232808', '122034419', '146131207', '145079757', '146051122', '144502474', '129443745', '133161138', '121102230', '144679540', '146175405', '142231145', '120279401', '146094202', '120541306', '145803217', '145271053', '128399671', '132241836', '146131625', '149042203', '122102729', '145511168', '109901026', '128665952', '128312539', '125342507', '132102113', '144149790', '146181021', '129016913', '145139725', '129583838', '146213719', '145221018', '148309156', '146262317', '109778162', '123094301', '129013658', '146215251', '146213107', '145199369', '121869713', '104567886', '148659625', '144509542', '146051558', '145519370', '146191836', '132043290', '117685128', '146233907', '130311644', '105916002', '145641130', '118014617', '146051643', '146112520', '121809649', '145069998', '146065211', '144508731', '145271713', '145341506', '133651

In [23]:
a = reg.loc[reg['zip_code'].astype(str).str.len() <= 5 ]
a['zip_code'].min()

10001

In [26]:
#load data and sheets
demand_class = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/Project 1 Data- version 0.0- Yue Chen/demand_classification.csv")
zips_cleaned = demand_class["zip_code_cleaned"]
populations = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/Project 1 Data- version 0.0- Yue Chen/population_calculated.csv")
#note, ignore children capacity and just use children capacity, as it is inconsistent ( could benefit from inspection)
regulated = pd.read_csv("/Users/phroog/Documents/Columbia/opt_models/proj1/Project 1 Data- version 0.0- Yue Chen/child_care_regulated_cleaned.csv")
#to regulated we are adding 0-5 and 5-12 columns
regulated["0-5_capacity"] = regulated["infant_capacity"] + regulated["toddler_capacity"] + regulated["preschool_capacity"]
#simple rename for 5-12, SUBJECT TO ADJUSTMENT
regulated["5-12_capacity"] = regulated["school_age_capacity"]

#HERE CHECK THAT 0-5 + 5-12 CAPCITIES ADD UP TO TOTAL_CAPACITY IN REGULATED DATAFRAME

In [29]:
for zc in zips_cleaned:
    try:
        r05 = populations[populations['zip_code_cleaned'] == zc]['Population_0_5'].iloc[0]
    except Exception as e:
        print(e)
        print(zc)
        break

single positional indexer is out-of-bounds
14428


In [32]:
d = populations[populations['zip_code_cleaned'] == 14428]
print(d)

Empty DataFrame
Columns: [zip_code_cleaned, Population_0_5, Population_0_12]
Index: []


In [34]:
# get unique zip sets
zips_pop = set(populations['zip_code_cleaned'])
zips_reg = set(regulated['zip_code_cleaned'])
zips_demand = set(demand_class['zip_code_cleaned'])

# zips in regulated but not in population
missing_in_pop = zips_reg - zips_pop

# zips in population but not in regulated
missing_in_reg = zips_pop - zips_reg

print(f"Zips in regulated but not in population: {len(missing_in_pop)}")
print(sorted(missing_in_pop))

print(f"\nZips in population but not in regulated: {len(missing_in_reg)}")
print(sorted(missing_in_reg))

### total discrepencies




Zips in regulated but not in population: 116
[10166, 11453, 11582, 12220, 12240, 12242, 12557, 13261, 14124, 14428, 14454, 14467, 14469, 14471, 14480, 14482, 14485, 14487, 14489, 14502, 14506, 14507, 14512, 14513, 14514, 14517, 14519, 14522, 14527, 14530, 14534, 14539, 14543, 14550, 14551, 14559, 14572, 14581, 14591, 14604, 14605, 14606, 14607, 14610, 14611, 14613, 14614, 14615, 14616, 14618, 14619, 14620, 14624, 14626, 14701, 14706, 14712, 14715, 14716, 14718, 14719, 14733, 14737, 14738, 14739, 14743, 14750, 14757, 14760, 14769, 14779, 14782, 14787, 14801, 14802, 14803, 14804, 14807, 14809, 14810, 14813, 14814, 14821, 14823, 14826, 14830, 14837, 14838, 14840, 14843, 14845, 14850, 14863, 14865, 14867, 14869, 14870, 14871, 14873, 14879, 14880, 14881, 14882, 14883, 14884, 14885, 14886, 14889, 14891, 14892, 14895, 14897, 14901, 14903, 14904, 14905]

Zips in population but not in regulated: 602
[6390, 10018, 10103, 10110, 10111, 10112, 10115, 10119, 10152, 10153, 10154, 10162, 10165, 10167